<a href="https://colab.research.google.com/github/spatank/Curiosity/blob/master/v8/persistent_homology_simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Curiosity_v8/')

Mounted at /content/drive


In [ ]:
!ls # run !ls to verify location

build_Wiki_networks.ipynb	persistent_homology_simulations.ipynb  Wiki
KNOT				persistent_homology_Wiki.ipynb
persistent_homology_KNOT.ipynb	Simulations


In [ ]:
import glob
import scipy
import scipy.io as sio
from scipy.io import savemat

In [ ]:
import sys
!{sys.executable} -m pip install Cython

In [ ]:
import sys
!{sys.executable} -m pip install Ripser

from ripser import ripser

     |████████████████████████████████| 71 kB 8.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 47 kB 4.6 MB/s 
  Created wheel for Ripser: filename=ripser-0.6.0-cp37-cp37m-linux_x86_64.whl size=454976 sha256=1970f9a1781844e2d7e287b5d10368662b04f0bc019ded07aafb89ff5277ec33
  Stored in directory: /root/.cache/pip/wheels/e8/b2/61/e9e1faaee99a8c232366e211456483ed0215c171f5b3db2f6f
  Created wheel for hopcroftkarp: filename=hopcroftkarp-1.2.5-py2.py3-none-any.whl size=18119 sha256=fa101a955f78cb3b8eccfdb9cc792f60acf05958934fceaa0221b90e8863da6f
  Stored in directory: /root/.cache/pip/wheels/d2/9f/a8/67f1b86e47cd17338d3d07939f4660378e65b758c4594f96e3
Successfully built Ripser hopcroftkarp


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_barcode(filt_mat, maxdim = 2):
    """
    Calculates the persistent homology for a given filtration matrix
    ``filt_mat``, default dimensions 0 through 2. Wraps ripser.
    """
    b = ripser(filt_mat, distance_matrix = True, maxdim = maxdim)['dgms']
    return list(zip(range(maxdim + 1), b))

def plot_barcode(bars, length, dims = [0, 1, 2], end = True):
    """
    Takes in bars, as generated by, e.g., ripser, and plots the barcode.
    """
    bars = dict(bars)
    count = 1
    has_inf = False
    colors = ['xkcd:emerald green', 'xkcd:tealish', 'xkcd:peacock blue']
    # iterate through dimension
    for d in dims:
        bn = bars[d]
        bn = sorted(bn, key = lambda x: x[0])
        for b, i in zip(bn, range(len(bn))):
            # extend in the case of infinite cycles
            if b[1] == np.inf:
                has_inf = True
                b = (b[0], 1.3*length)
            # plot first one with label
            if i == 0:
                plt.plot(b, [count, count], color = colors[d],
                         label='{}-cycles'.format(d))
            else:
                plt.plot(b, [count, count], color = colors[d])
            count += 1
        count += 1
    # add end of filtration line
    plt.axvline(x = length, color = 'xkcd:grey', alpha = 0.5, linestyle = ':')
    if end:
        plt.annotate('Filtration End', (length + 10, 0.5 * count), rotation = 270,
                     color = 'xkcd:grey', alpha = 0.5)
    lims = plt.xlim()
    plt.xlim([-0.05 * length, length * 1.05])
    plt.xlabel('Nodes')
    plt.ylabel('Cycle Number')

def betti_curves(bars, length):
    """
    Takes in bars and returns the betti curves
    """
    bettis = np.zeros((len(bars), length))
    for i in range(bettis.shape[0]):
        bn = bars[i][1]
        for bar in bn:
            birth = int(bar[0])
            death = length+1 if np.isinf(bar[1]) else int(bar[1]+1)
            bettis[i][birth:death] += 1
    return bettis

In [ ]:
mat_file = '/content/drive/My Drive/Curiosity_v8/Simulations/simulated_nets.mat'
mat_contents = sio.loadmat(mat_file)

CP_nets = mat_contents['const_prob_nets']
CP_nets_weighted = mat_contents['const_prob_nets_weighted']
PP_nets = mat_contents['prop_prob_nets']
PP_nets_weighted = mat_contents['prop_prob_nets_weighted']
PA_nets = mat_contents['PA_nets']
PA_nets_weighted = mat_contents['PA_nets_weighted']

num_nodes = CP_nets_weighted.shape[0]
num_iters = CP_nets_weighted.shape[2]

In [ ]:
for iter in range(num_iters):

  print('Beginning iter. %d' % iter)
  
  # Run persistent homology for `constant probability' networks
  bettis_0_CP = []
  bettis_1_CP = []
  bettis_2_CP = []
  for i in range(num_iters):
    bars_CP = get_barcode(CP_nets_weighted[:, :, i])
    bettis_CP = betti_curves(bars_CP, num_nodes)
    bettis_0_CP.append(bettis_CP[0])
    bettis_1_CP.append(bettis_CP[1])
    bettis_2_CP.append(bettis_CP[2])

  # Run persistent homology for `proportional probability' networks
  bettis_0_PP = []
  bettis_1_PP = []
  bettis_2_PP = []
  for i in range(num_iters):
    bars_PP = get_barcode(PP_nets_weighted[:, :, i])
    bettis_PP = betti_curves(bars_PP, num_nodes)
    bettis_0_PP.append(bettis_PP[0])
    bettis_1_PP.append(bettis_PP[1])
    bettis_2_PP.append(bettis_PP[2])

  # Run persistent homology for `preferential attachment' networks
  bettis_0_PA = []
  bettis_1_PA = []
  bettis_2_PA = []
  for i in range(num_iters):
    bars_PA = get_barcode(PA_nets_weighted[:, :, i])
    bettis_PA = betti_curves(bars_PA, num_nodes)
    bettis_0_PA.append(bettis_PA[0])
    bettis_1_PA.append(bettis_PA[1])
    bettis_2_PA.append(bettis_PA[2])

  print('Processed iter. %d' % iter)

base_path = '/content/drive/My Drive/Curiosity_v8/Simulations/'

# Store all PH results (one example barcode for each network model)
save_dict = {'num_nodes': num_nodes,
             'bars_CP': bars_CP,
             'bettis_0_CP': bettis_0_CP, 
             'bettis_1_CP': bettis_1_CP,
             'bettis_2_CP': bettis_2_CP,
             'bars_PP': bars_PP,
             'bettis_0_PP': bettis_0_PP,
             'bettis_1_PP': bettis_1_PP,
             'bettis_2_PP': bettis_2_PP,
             'bars_PA': bars_PA,
             'bettis_0_PA': bettis_0_PA,
             'bettis_1_PA': bettis_1_PA,
             'bettis_2_PA': bettis_2_PA}
             
np.save(base_path + 'simulated_nets_PH.npy', save_dict, allow_pickle = True) 
savemat(base_path + 'simulated_nets_PH.mat', save_dict)

Beginning iter. 0
Processed iter. 0
Beginning iter. 1
Processed iter. 1
Beginning iter. 2
Processed iter. 2
Beginning iter. 3
Processed iter. 3
Beginning iter. 4
Processed iter. 4
Beginning iter. 5
Processed iter. 5
Beginning iter. 6
Processed iter. 6
Beginning iter. 7
Processed iter. 7
Beginning iter. 8
Processed iter. 8
Beginning iter. 9
Processed iter. 9
Beginning iter. 10
Processed iter. 10
Beginning iter. 11
Processed iter. 11
Beginning iter. 12
Processed iter. 12
Beginning iter. 13
Processed iter. 13
Beginning iter. 14
Processed iter. 14
Beginning iter. 15
Processed iter. 15
Beginning iter. 16
Processed iter. 16
Beginning iter. 17
Processed iter. 17
Beginning iter. 18
Processed iter. 18
Beginning iter. 19
Processed iter. 19
Beginning iter. 20
Processed iter. 20
Beginning iter. 21
Processed iter. 21
Beginning iter. 22
Processed iter. 22
Beginning iter. 23
Processed iter. 23
Beginning iter. 24
Processed iter. 24
Beginning iter. 25
Processed iter. 25
Beginning iter. 26
Processed ite

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
